In [1]:
import pandas as pd
import numpy as np
import matplotlib
import datetime
import time

matplotlib.use("Agg")
import matplotlib.pyplot as plt


def scraping(stock, days):

	end = datetime.date.today()
	start = end - datetime.timedelta(days=days)

	base = "http://info.finance.yahoo.co.jp/history/?code={0}.T&{1}&{2}&tm={3}&p={4}"

	start_g = str(start)
	start_g = start_g.split("-")
	start_g = "sy={0}&sm={1}&sd={2}".format(start_g[0], start_g[1], start_g[2])
	end = str(end)
	end = end.split("-")
	end = "ey={0}&em={1}&ed={2}".format(end[0], end[1], end[2])
	page = 1
	term = "d"
	#term = "m"

	result = []
	while True:
		url = base.format(stock, start_g, end, term, page)
		# print(url)
		try:
			df = pd.read_html(url, header=0)
		except ValueError:
			break

		if len(df[1]) == 0:
			break

		result.append(df[1])
		page += 1
		time.sleep(1.0)


	try:
		result = pd.concat(result)
		result.columns = ["Date", "Open", "High", "Low", "Close", "Volume", "Adj Close"]
		
		yahoo_df = pd.DataFrame(result)
		yahoo_df.to_csv("./data/code_" + str(stock) + ".csv", encoding="utf-8")

	except ValueError:
		pass


In [2]:
stock_list = [7148, 9024,6504]
days = 180
for s in stock_list:
	scraping(s, days)

In [3]:
master_df = pd.DataFrame({})
for s in stock_list:
	df = pd.read_csv("./data/code_{0}.csv".format(s))
	df = df.sort_values("Date", ascending=True)
	print(df.tail(10))

	tmp_df = df.loc[:, ["Date", "Adj Close"]]
	tmp_df.columns = ["Date_{0}".format(s), "Close_{0}".format(s)]

	master_df = pd.concat([master_df, tmp_df["Close_{0}".format(s)]], axis=1)

    Unnamed: 0        Date  Open  High   Low  Close   Volume  Adj Close
25           5   2018年8月8日  1129  1154  1119   1145   651400       1145
24           4   2018年8月9日  1150  1153  1132   1132   514100       1132
2            2  2018年9月10日  1138  1144  1132   1134   410500       1134
1            1  2018年9月11日  1124  1124  1094   1114  1271400       1114
0            0  2018年9月12日  1110  1115  1074   1079  1050800       1079
7            7   2018年9月3日  1218  1219  1171   1173   943800       1173
6            6   2018年9月4日  1187  1191  1167   1174   601900       1174
5            5   2018年9月5日  1174  1179  1162   1164   451900       1164
4            4   2018年9月6日  1155  1156  1143   1147   578800       1147
3            3   2018年9月7日  1145  1145  1121   1141   670000       1141
    Unnamed: 0        Date  Open  High   Low  Close   Volume  Adj Close
25           5   2018年8月8日  2052  2065  2035   2041  1337600       2041
24           4   2018年8月9日  2020  2025  1995   2019   960500    

In [9]:
# Calculate annual average return # 年率を計算する前に日率を計算
# .pct_change：Rate of change # リターンを計算する便利な関数
print(master_df.tail(10))
returns_daily = master_df.pct_change()
print(returns_daily.tail(10))

# Convert to annual rate. working day is 250/365. # 年率に変換. 250は年間の市場が動いている日数.
returns_annual = returns_daily.mean() * 250
print("returns_annual")
print(returns_annual)

# Calculate covariance # 各々のリターンから共分散を計算
cov_daily = returns_daily.cov()
cov_annual = cov_daily * 250
print("cov_annual")
print(cov_annual)

    Close_7148  Close_9024  Close_6504
25        1145        2041         861
24        1132        2019         863
2         1134        2002         861
1         1114        1995         862
0         1079        2017         860
7         1173        2006         885
6         1174        2018         879
5         1164        2002         879
4         1147        1969         882
3         1141        1999         877
    Close_7148  Close_9024  Close_6504
25    0.013274    0.000490    0.014134
24   -0.011354   -0.010779    0.002323
2     0.001767   -0.008420   -0.002317
1    -0.017637   -0.003497    0.001161
0    -0.031418    0.011028   -0.002320
7     0.087118   -0.005454    0.029070
6     0.000853    0.005982   -0.006780
5    -0.008518   -0.007929    0.000000
4    -0.014605   -0.016484    0.003413
3    -0.005231    0.015236   -0.005669
returns_annual
Close_7148   -0.210908
Close_9024    0.239851
Close_6504    0.382434
dtype: float64
cov_annual
            Close_7148  Close_90

In [5]:
# Define list of returns, volatilities, ratiosm, sharpe ratio # 諸々のリスト定義
port_returns = []
port_volatility = []
stock_weights = []
sharpe_ratio = []

# Choose a prime number. # seedは素数を選ぼう
np.random.seed(101)

# Number of stocks to be combined # 組み入れる株の数
num_assets = len(stock_list)

# Number of trials of portfolio creation pattern # ポートフォリオ作成パターンの試行回数
num_portfolios = 50000

# Randomly calculate portfolio risks and returns # 様々な銘柄の比率でのポートフォリオのリターンとリスクを計算
for single_portfolio in range(num_portfolios):

	# Determination of the ratio of stocks by random number # 銘柄の比率を乱数で決定
	weights = np.random.random(num_assets)
	weights /= np.sum(weights)

	# Calculate expected return on portfolio # ポートフォリオの期待リターンを計算
	returns = np.dot(weights, returns_annual)
	# print("returns : ", returns)

	# Calculate portfolio volatility # ポートフォリオのボラティリティを計算
	volatility = np.sqrt(np.dot(weights.T, np.dot(cov_annual, weights)))
	# print("volatility : ", volatility)

	# Calculate sharp ratio
	sharpe = returns / volatility
	sharpe_ratio.append(sharpe)

	# Store calculated values in list # 計算値をリストに格納
	port_returns.append(returns)
	port_volatility.append(volatility)
	stock_weights.append(weights)

# a dictionary for Returns and Risk values of each portfolio # 辞書型に格納
portfolio = {"Returns": port_returns,
			"Volatility": port_volatility,
			"Sharpe Ratio" : sharpe_ratio}

# Add ratio data # 計算したポートフォリオのリターンとリスクに、比率のデータを加える
for counter,symbol in enumerate(stock_list):
    portfolio[str(symbol) + " Weight"] = [Weight[counter] for Weight in stock_weights]

# Convert to DataFrame # PandasのDataFrameに変換
df = pd.DataFrame(portfolio)

# Done # データフレーム完成
column_order = ["Returns", "Volatility", "Sharpe Ratio"] + [str(stock)+" Weight" for stock in stock_list]
df = df[column_order]

print(df.head(10))

    Returns  Volatility  Sharpe Ratio  7148 Weight  9024 Weight  6504 Weight
0  0.034828    0.274581      0.126840     0.462913     0.511562     0.025525
1  0.264447    0.290182      0.911316     0.101450     0.405323     0.493227
2  0.221388    0.274845      0.805503     0.159702     0.464909     0.375389
3  0.207553    0.271420      0.764692     0.173255     0.505544     0.321201
4  0.268652    0.291545      0.921477     0.094100     0.406421     0.499480
5  0.219535    0.329833      0.665594     0.252706     0.090881     0.656414
6  0.103275    0.300268      0.343943     0.431219     0.163403     0.405378
7 -0.063712    0.354074     -0.179940     0.735376     0.068848     0.195775
8  0.233952    0.280290      0.834680     0.109855     0.584222     0.305923
9  0.140895    0.266365      0.528954     0.311920     0.396007     0.292073


In [6]:
# look an efficient frontier # 50000パターンのポートフォリオを可視化して、効率的フロンティアを探る
plt.style.use("seaborn")
df.plot.scatter(x="Volatility", y="Returns", c="Sharpe Ratio", cmap="RdYlGn", edgecolors="black", figsize=(10, 8), grid=True)
plt.xlim([0,1])
plt.xlabel("Volatility (Std. Deviation)")
plt.ylabel("Expected Returns")
plt.title("Efficient Frontier")
plt.savefig("image3.png")


In [7]:
# Calculate sharp ratio
sharpe = returns / volatility
sharpe_ratio.append(sharpe)

In [8]:
df = df.sort_values("Sharpe Ratio", ascending=True)
print(df.tail(1))

       Returns  Volatility  Sharpe Ratio  7148 Weight  9024 Weight  \
19368  0.32537    0.314609      1.034203     0.000033     0.400082   

       6504 Weight  
19368     0.599885  
